#### Handling Nulls

- dropping nulls
- filling nulls

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = spark.read.format('csv').option('inferSchema', True)\
        .option('header', True)\
            .load('/Volumes/workspace/default/pyspark/employee_new.csv')

df.display()

In [0]:
### Dropping Nulls


df.dropna('all').display()

In [0]:
df.display()

In [0]:
df.dropna('any').display()

In [0]:
df.dropna(subset = ['department']).display()

In [0]:
#### Filling Nulls

df.fillna('NA').display()

df.withColumn('joining_date', col('joining_date').cast(StringType())).fillna('NA', subset = ['joining_date']).display()
#df.fillna('NA', subset = ['joining_date']).display()


##### Split and indexing function

In [0]:
df_new = df.withColumn('Role', lit('Data Engineer'))

df_new.display()

In [0]:
df_new.withColumn('Role', split('Role', ' ')).display() ### Spliting

df_new.withColumn('Role', split('Role', ' ')[1]).display()  ### Indexing


#### EXPLODE Function

In [0]:
df_expld = df_new.withColumn('Role', split('Role', ' '))

df_expld.display()

In [0]:
df_expld.withColumn('Role', explode('Role')).display()




#### ARRAY_CONTAINS FUNCTION

In [0]:
df_expld.display()


In [0]:
df_expld.withColumn('role_new', array_contains('Role', 'Data')).display()


#### GROUP_BY FUNCTION

In [0]:
df_expld.groupBy('department').sum('salary').display()



df_expld.groupBy('department').agg(sum('salary')).display()

In [0]:
df_expld.groupBy('department').avg('salary').display()

df_expld.groupBy('department').agg(avg('salary')).display()

In [0]:
df_new.display()

df_grpby = df_new.withColumn('Role', when((col("department") == "HR") & (col("employee_id") == 101), "Talent Management")
    .otherwise(col("Role")))

df_grpby = df_grpby.withColumn('Role', when((col("department") == "HR") & (col("employee_id") == 106), "Talent Aquisition")
    .otherwise(col("Role")))

df_grpby = df_grpby.fillna('IT', subset = ['department'])

df_grpby = df_grpby.withColumn('Role', when((col("department") == "IT") & (col("employee_id") == 104), "Data Scientist")
    .otherwise(col("Role")))

df_grpby.display()

In [0]:
df_grpby.groupBy('department', 'Role').agg(sum('salary')).display()


##### collect_list function

In [0]:
df_grpby.groupBy('department').agg(collect_list('Role')).display()



###PIVOT Function

In [0]:
df_grpby.groupBy('department').pivot('Role').agg(sum('salary')).display()


#### WHEN-OTHERWISE Function

In [0]:
#df_grpby.display()

df_con = df_grpby.withColumn('Vacancy', when(col('department') == 'IT', 'Yes').otherwise('No'))

df_con.display()

In [0]:
df_cond = df_grpby.withColumn('new_vacancy', when(((col('department') == 'IT') & (col('Role') == 'Data Engineer')), 'Active')\
    .when(((col('department') == 'HR') & (col('Role') == 'Talent Management')), 'Needed')\
    .otherwise('Inactive'))

df_cond.display()